In [69]:
import pandas as pd

# Load the CSV file, specifying the semicolon delimiter
df = pd.read_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/dataset.csv')


C:\Users\Destiny\AppData\Local\Temp\ipykernel_20780\619231167.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/dataset.csv')


In [19]:
df.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,NaN,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,NaN,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
#create DF with artist info

# Step 1: Split the 'artist' column by semicolon and explode into separate rows
df_split = df['artists'].str.split(';', expand=True)

df_split = df['artists'].str.split(';', expand=True)

df_split.columns = [f'artist_{i+1}' for i in range(df_split.shape[1])]

df_final = pd.concat([df, df_split], axis=1)

unique_artists = df['artists'].str.split(';', expand=True).stack().unique()
unique_artists_df = pd.DataFrame(unique_artists, columns=['artists'])

#df_final.head()


In [ ]:
#Edit DF to split up delimited artists

# Ensure the 'artists' column is string type
df['artists'] = df['artists'].astype(str)

# Split the 'artists' column into lists where necessary
df['artists'] = df['artists'].apply(lambda x: x.split(';') if ';' in x else [x])

# Explode the 'artists' column to create one row per artist
df_exploded = df.explode('artists').reset_index(drop=True)

# Strip any leading/trailing whitespace from artist names
df_exploded['artists'] = df_exploded['artists'].str.strip()

print(df_exploded.head())

In [ ]:
import requests
import json
import pandas as pd
import time

Error fetching data for The Travelin' McCourys: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [136]:
'''Additional artist level data using AudioDB ID'''

API_KEY = '523532'

#Get Artist AudioDB ID using their name
def get_artist_info(artist_name):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/search.php?s={artist_name}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and data['artists'] is not None:
                    artist = data['artists'][0]
                    return artist.get('idArtist', 'N/A'), artist.get('strLabel', 'N/A')
                else:
                    #if response was successful but artist doesnt exist
                    return 'N/A','N/A'
            else:
                #if I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_name}: {e}")
    return 'N/A', 'N/A'

#Get Artist details by their AudioDB ID
def get_artist_details(artist_id):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/artist.php?i={artist_id}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and data['artists'] is not None:
                    artist = data['artists'][0]
                    return(
                        artist.get('strStyle', 'N/A'),
                        artist.get('strGenre', 'N/A'),
                        artist.get('strMood', 'N/A'),
                        artist.get('strFacebook', 'N/A'),
                        artist.get('strTwitter', 'N/A'),
                        artist.get('strGender', 'N/A'),
                        artist.get('strMusicBrainzID', 'N/A'),
                        artist.get('strLastFMChart', 'N/A')
                    )
                else:
                    #if response was successful but artist doesnt exist
                    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')
            else:
                #if I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_id}: {e}")
    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')



#Social media data:
def get_artist_socials(artist_id): 
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/artist-links.php?i={artist_id}"
    
    #only want to pull info for these social medias:
    social_media = {
        'facebook': '',
        'twitter': '',
        'spotify': '',
        'deezer': '',
        'itunes': '',
        'tidal': '',
        'youtube': '',
        'instagram': ''
    }
    
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            #print(f"Received response. Status code: {response.status_code}") 

            if response.status_code == 200:
                data = response.json()
                #print(f"JSON data received: {data}")  # Debug print

                if 'artists' in data and data['artists'] is not None:
                    for link in data['artists']:
                        if link['strSite'] in social_media:
                            social_media[link['strSite']] = link['strLink']
                    return(social_media)
                else:
                    print(f"No social media data found for artist ID:  {artist_id}")
                    return (social_media)
            else:
                #if I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_id}: {e}")
        
    #print(f"Returning social media data: {social_media}")  # Debug print
    return(social_media)

In [ ]:
'''Create table of Artist level data using API functions'''

#Create two new columns to the artist data table
unique_artists_df['artist_id'] = [None] * len(unique_artists_df)
unique_artists_df['record_label'] = [None] * len(unique_artists_df)

# Apply the function to get data for two new columns
for index in range(len(unique_artists_df)):
    myartist = unique_artists_df['artists'][index]
    #print("processing:",myartist)
    callresponse = get_artist_info(myartist)
    unique_artists_df['artist_id'][index] = callresponse[0]
    unique_artists_df['record_label'][index] = callresponse[1]
    # Add a 1-second delay between API calls
    #time.sleep(1)

In [ ]:
'''Add additional Artist level data to artist table'''

unique_artists_details = unique_artists_df.copy()

#filter for only those with an audioDB id
unique_artists_details = unique_artists_details[unique_artists_details['artist_id']!='N/A']

# Add new columns for the API results
unique_artists_details['strStyle'] = 'N/A'
unique_artists_details['strGenre'] = 'N/A'
unique_artists_details['strMood'] = 'N/A'
unique_artists_details['strFacebook'] = 'N/A'
unique_artists_details['strTwitter'] = 'N/A'
unique_artists_details['strGender'] = 'N/A'
unique_artists_details['strMusicBrainzID'] = 'N/A'
unique_artists_details['strLastFMChart'] = 'N/A'

# Apply the function to create two new columns
for index, row in unique_artists_details.iterrows():
    artist_id = row['artist_id']
    callresponse = get_artist_details(artist_id)
    
    # Update the DataFrame with API results
    unique_artists_details.at[index, 'strStyle'] = callresponse[0]
    unique_artists_details.at[index, 'strGenre'] = callresponse[1]
    unique_artists_details.at[index, 'strMood'] = callresponse[2]
    unique_artists_details.at[index, 'strFacebook'] = callresponse[3]
    unique_artists_details.at[index, 'strTwitter'] = callresponse[4]
    unique_artists_details.at[index, 'strGender'] = callresponse[5]
    unique_artists_details.at[index, 'strMusicBrainzID'] = callresponse[6]
    unique_artists_details.at[index, 'strLastFMChart'] = callresponse[7]

# Display the updated DataFrame & check we have all data
#print(len(unique_artists_details))
print(unique_artists_details.head(5))

             artists artist_id record_label          strStyle strGenre  \
2  Ingrid Michaelson    120114         None              Folk    Indie   
3               ZAYN    141655     Columbia          Rock/Pop      Pop   
4       Kina Grannis    111830         None          Rock/Pop      Pop   
6       Tyrone Wells    147082         None              None     Folk   
7  A Great Big World    126846         Epic  Manufactured Pop      Pop   

        strMood                     strFacebook strTwitter strGender  \
2                                                        1    Female   
3  Good Natured               facebook.com/zayn          1      Male   
4                en-gb.facebook.com/kinagrannis          1    Female   
6          None                            None       None      None   
7         Happy                                          1      Male   

                       strMusicBrainzID  \
2  1fc494a1-9109-4081-a455-2d05bea9d2bf   
3  985f7e6f-0a7e-4de7-b9ec-a5dac63cb

In [138]:
'''Get artist social media data and add it to artist table'''
unique_artists_social_media = unique_artists_df.copy()

#filter for only those with an audioDB id
unique_artists_social_media = unique_artists_social_media[unique_artists_social_media['artist_id']!='N/A']

# Add new columns for the API results
unique_artists_social_media['twitter_link'] = [None] * len(unique_artists_social_media)
unique_artists_social_media['spotify_link'] = [None] * len(unique_artists_social_media)
unique_artists_social_media['deezer_link']  = [None] * len(unique_artists_social_media)
unique_artists_social_media['itunes_link']  = [None] * len(unique_artists_social_media)
unique_artists_social_media['tidal_link']   = [None] * len(unique_artists_social_media)
unique_artists_social_media['youtube_link'] = [None] * len(unique_artists_social_media)
unique_artists_social_media['instagram_link'] = [None] * len(unique_artists_social_media)


# Apply the function to create two new columns
for index, row in unique_artists_social_media.iterrows():
#for index, row in tester.iterrows():

    artist_id = row['artist_id']
    callresponse = get_artist_socials(artist_id) #this is a dictionary now

    #print(callresponse)
    
    #loop through the dictionary:
    for platform, link in callresponse.items(): 
        # Update the DataFrame with API results
        unique_artists_social_media.at[index, f'{platform}_link'] = link
        #tester.at[index, f'{platform}_link'] = link

# Display the updated DataFrame & check we have all data
print(len(unique_artists_social_media))
print(unique_artists_social_media.head(5))
#print(tester)


No social media data found for artist ID:  147082
No social media data found for artist ID:  177987
No social media data found for artist ID:  159291
No social media data found for artist ID:  133184
No social media data found for artist ID:  139910
No social media data found for artist ID:  145009
No social media data found for artist ID:  137006
No social media data found for artist ID:  133979
No social media data found for artist ID:  135073
No social media data found for artist ID:  136689
No social media data found for artist ID:  143164
No social media data found for artist ID:  165381
No social media data found for artist ID:  139688
No social media data found for artist ID:  157730
No social media data found for artist ID:  139503
No social media data found for artist ID:  179786
No social media data found for artist ID:  169341
No social media data found for artist ID:  162401
No social media data found for artist ID:  128427
No social media data found for artist ID:  153008


In [161]:
'''Track level data:'''
def get_track_details(artist_name, track_name):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/searchtrack.php?s={artist_name}&t={track_name}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'track' in data and data['track'] is not None:
                    track = data['track'][0]
                    return(
                        track.get('idTrack', 'N/A'),
                        track.get('strTheme', 'N/A'),
                        track.get('strMusicVid', 'N/A'),
                        track.get('intMusicVidViews', 'N/A'),
                        track.get('intMusicVidLikes', 'N/A'),
                        track.get('intMusicVidDislikes', 'N/A'),
                        track.get('intMusicVidComments', 'N/A'),
                        track.get('intTotalListeners', 'N/A'),
                        track.get('intTotalPlays', 'N/A'),
                        track.get('strMusicBrainzID', 'N/A') #probably dont need but just in case
                    )
                else:
                    #if response was successful but artist doesnt exist
                    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')
            else:
                #if I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_name} and {track_name}: {e}")
    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')


In [ ]:
'''Add Track level data'''
#You can use this or the code chunk below which has been updated to run faster than this code chunk. 

track_data = df_exploded.copy()
#tester = track_data.head(5).copy() 

#Create new columns to the artist data table
track_data['idTrack']      = [None] * len(track_data)
track_data['strTheme']     = [None] * len(track_data)
track_data['strMusicVid']  = [None] * len(track_data)
track_data['intMusicVidViews']  = [None] * len(track_data)
track_data['intMusicVidLikes']  = [None] * len(track_data)
track_data['intMusicVidDislikes'] = [None] * len(track_data)
track_data['intMusicVidComments'] = [None] * len(track_data)
track_data['intTotalListeners']  = [None] * len(track_data)
track_data['intTotalPlays'] = [None] * len(track_data)
track_data['strMusicBrainzID'] = [None] * len(track_data)

# Apply the function to get data for two new columns
#for index, row in tester.iterrows():
for index, row in track_data.iterrows():
    artistname = track_data['artists'][index]
    #artistname = tester['artists'][index]
    trackname = track_data['track_name'][index]
    #trackname = tester['track_name'][index]
    #print("processing:",artistname,",",trackname)
    if index % 1000 == 0:
        print(f"Reached the {index}th iteration")
    callresponse = get_track_details(artistname, trackname)
    track_data.loc[index, 'idTrack']        = callresponse[0]
    track_data.loc[index, 'strTheme']       = callresponse[1]
    track_data.loc[index, 'strMusicVid']    = callresponse[2]
    track_data.loc[index, 'intMusicVidViews']    = callresponse[3]
    track_data.loc[index, 'intMusicVidLikes']    = callresponse[4]
    track_data.loc[index, 'intMusicVidDislikes'] = callresponse[5]
    track_data.loc[index, 'intMusicVidComments'] = callresponse[6]
    track_data.loc[index, 'intTotalListeners']   = callresponse[7]
    track_data.loc[index, 'intTotalPlays']       = callresponse[8]
    track_data.loc[index, 'strMusicBrainzID']    = callresponse[9]
    # tester.loc[index, 'idTrack']        = callresponse[0]
    # tester.loc[index, 'strTheme']       = callresponse[1]
    # tester.loc[index, 'strMusicVid']    = callresponse[2]
    # tester.loc[index, 'intMusicVidViews']    = callresponse[3]
    # tester.loc[index, 'intMusicVidLikes']    = callresponse[4]
    # tester.loc[index, 'intMusicVidDislikes'] = callresponse[5]
    # tester.loc[index, 'intMusicVidComments'] = callresponse[6]
    # tester.loc[index, 'intTotalListeners']   = callresponse[7]
    # tester.loc[index, 'intTotalPlays']       = callresponse[8]
    # tester.loc[index, 'strMusicBrainzID']    = callresponse[9]

    # Add a 1-second delay between API calls
    #time.sleep(1)

Reached the 0th iteration


KeyboardInterrupt: 

In [ ]:
'''Add Track Level Data'''
#same as above code chunk. Just updated to run faster 

from concurrent.futures import ThreadPoolExecutor, as_completed

# Make a copy of the original DataFrame
track_data = df_exploded.copy()

# Add new columns
columns_to_add = [
    'idTrack', 'strTheme', 'strMusicVid', 'intMusicVidViews', 'intMusicVidLikes',
    'intMusicVidDislikes', 'intMusicVidComments', 'intTotalListeners',
    'intTotalPlays', 'strMusicBrainzID'
]
for col in columns_to_add:
    track_data[col] = None

# Function wrapper to include index for mapping back
def fetch_track_data(index, artistname, trackname):
    try:
        result = get_track_details(artistname, trackname)
        return index, result
    except Exception as e:
        print(f"Error at index {index}: {e}")
        return index, [None] * 10  # default error values

# Prepare input list
inputs = [(index, row['artists'], row['track_name']) for index, row in track_data.iterrows()]

# Run in batches of 10
batch_size = 10
for i in range(0, len(inputs), batch_size):

    print(f"Processing batch {i} to {i+batch_size}")
    batch = inputs[i:i + batch_size]

    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        futures = [executor.submit(fetch_track_data, *args) for args in batch]

        for future in as_completed(futures):
            index, data = future.result()
            if data:
                for col, value in zip(columns_to_add, data):
                    track_data.at[index, col] = value


Processing batch 0 to 10
Processing batch 10 to 20
Processing batch 20 to 30
Processing batch 30 to 40
Processing batch 40 to 50
Processing batch 50 to 60
Processing batch 60 to 70
Processing batch 70 to 80
Processing batch 80 to 90
Processing batch 90 to 100
Processing batch 100 to 110
Processing batch 110 to 120
Processing batch 120 to 130
Processing batch 130 to 140
Processing batch 140 to 150
Processing batch 150 to 160
Processing batch 160 to 170
Processing batch 170 to 180
Processing batch 180 to 190
Processing batch 190 to 200
Processing batch 200 to 210
Processing batch 210 to 220
Processing batch 220 to 230
Processing batch 230 to 240
Processing batch 240 to 250
Processing batch 250 to 260
Processing batch 260 to 270
Processing batch 270 to 280
Processing batch 280 to 290
Processing batch 290 to 300
Processing batch 300 to 310
Processing batch 310 to 320
Processing batch 320 to 330
Processing batch 330 to 340
Processing batch 340 to 350
Processing batch 350 to 360
Processing b

In [ ]:
#Validate the track level data pull from cell above

track_data[(track_data['strMusicVid'] != 'N/A') & (track_data['strMusicVid'].notna())]
#track_data.to_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/track_data.csv')


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,idTrack,strTheme,strMusicVid,intMusicVidViews,intMusicVidLikes,intMusicVidDislikes,intMusicVidComments,intTotalListeners,intTotalPlays,strMusicBrainzID
7,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.147,...,33787651,...,https://www.youtube.com/watch?v=-2U0Ivkn2Ds,587530613,4217070,None,128910,374047,2678022,fcd9bec7-5a8f-48fb-a90f-bf8d22010f35
9,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.444,...,32761119,...,https://www.youtube.com/watch?v=EkHTsc9PU2A,460495817,2265897,51838,104304,1292172,10404068,a5a2330e-2fff-4601-a715-6e68a8e98fbf
10,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.414,...,32761120,...,https://www.youtube.com/watch?v=acvIVA9-FMQ,180282677,1001308,None,31575,315748,2173377,f68a7497-91aa-4d8a-88c8-abc0ba5b49f7
13,10,4mzP5mHkRvGxdhdGdAH7EJ,Zack Tabudlo,Episode,Give Me Your Forever,74,244800,False,0.627,0.363,...,36277303,...,https://www.youtube.com/watch?v=BKenhu5Nph4,7739169,150017,None,3373,None,None,a7e514a7-e25b-45aa-889a-d61a55076afb
14,11,5ivF4eQBqJiVL5IAE9jRyl,Jason Mraz,Love Is a Four Letter Word,I Won't Give Up,69,240165,False,0.483,0.303,...,32761172,None,https://www.youtube.com/watch?v=O1-4u9W-bns,68919201,393728,5613,23983,406192,3048501,7a49cb65-1f22-4334-b139-6500c2e79ee5
24,18,2qLMf6TuEC3ruGJg4SMMN6,Jason Mraz,We Sing. We Dance. We Steal Things.,Lucky,68,189613,False,0.625,0.414,...,32761120,...,https://www.youtube.com/watch?v=acvIVA9-FMQ,180282677,1001308,None,31575,315748,2173377,f68a7497-91aa-4d8a-88c8-abc0ba5b49f7
28,20,3S0OXQeoh0w6AY8WQVckRW,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,75,242946,False,0.703,0.444,...,32761119,...,https://www.youtube.com/watch?v=EkHTsc9PU2A,460495817,2265897,51838,104304,1292172,10404068,a5a2330e-2fff-4601-a715-6e68a8e98fbf
31,22,5TvE3pk05pyFIGdSY9j4DJ,A Great Big World,Is There Anybody Out There? - Track by Track C...,Say Something,70,229400,False,0.407,0.147,...,33787651,...,https://www.youtube.com/watch?v=-2U0Ivkn2Ds,587530613,4217070,None,128910,374047,2678022,fcd9bec7-5a8f-48fb-a90f-bf8d22010f35


In [ ]:
'''Combine datasets to have one artist level file and one track/artist level file '''
all_artists = unique_artists_df.merge(unique_artists_details, on = ["artists", "artist_id", "record_label"], how = "left")
all_artists = all_artists.merge(unique_artists_social_media, on = ['artists', 'artist_id', 'record_label'], how = "left")

# Now merge with the artist information table
#original_db_additional_columns = df_exploded.merge(all_artists, on='artists', how='left')
#original_db_additional_columns.to_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/original_db_additional_columns.csv')

#unique_artists_social_media.to_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/social_media.csv')
#all_artists.to_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/Artist_Details.csv')
#unique_artists_details.to_csv('C:/Users/Destiny/Documents/2. School/Graduate/CSE6242/FINAL PROJECT/artistdetails.csv')

# import sqlite3
# connection = sqlite3.connect('mydatabase.db') 
# cursor = connection.cursor()
# unique_artists_df.to_sql('Artist_info', connection, if_exists='replace', index=False)
# connection.commit()
# connection.close()


,artists,artist_id,record_label,strStyle,strGenre,strMood,strFacebook,strTwitter,strGender,strMusicBrainzID,strLastFMChart,twitter_link,spotify_link,deezer_link,itunes_link,tidal_link,youtube_link,instagram_link,facebook_link
0,Gen Hoshino,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ben Woodward,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ingrid Michaelson,120114,None,Folk,Indie,,,1,Female,1fc494a1-9109-4081-a455-2d05bea9d2bf,None,twitter.com/ingridmusic,open.spotify.com/artist/2vm8GdHyrJh2O2MfbQFYG0,www.deezer.com/artist/16797,itunes.apple.com/us/artist/id157692029,listen.tidal.com/artist/62450,www.youtube.com/user/IngridMichaelson,www.instagram.com/ingridmichaelson/,www.facebook.com/ingridmichaelson
3,ZAYN,141655,Columbia,Rock/Pop,Pop,Good Natured,facebook.com/zayn,1,Male,985f7e6f-0a7e-4de7-b9ec-a5dac63cb2f7,None,twitter.com/zaynmalik,open.spotify.com/artist/5ZsFI1h6hIdQRw2ti0hz81,www.deezer.com/artist/9761322,itunes.apple.com/gb/artist/id973181994,,www.youtube.com/channel/UC3PdiRW5dUA4V70ueeR1eHA,www.instagram.com/zayn/,facebook.com/zayn
4,Kina Grannis,111830,None,Rock/Pop,Pop,,en-gb.facebook.com/kinagrannis,1,Female,7a1766ba-3d50-428f-b6f5-4d267248c744,http://www.last.fm/music/Kina Grannis/+charts?...,twitter.com/kinagrannis,open.spotify.com/artist/7h4j9YTJJuAHzLCc3KCvYu,www.deezer.com/artist/405995,itunes.apple.com/artist/id415821967,listen.tidal.com/artist/3894285,www.youtube.com/user/kinagrannis,www.instagram.com/kinagrannis/,en-gb.facebook.com/kinagrannis
